In [ ]:
# Import necessary libraries
import tensorflow as tf
from tensorflow.keras.applications import ResNet101
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.mixed_precision import set_global_policy
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.utils.class_weight import compute_class_weight
import os

In [ ]:

# Enable mixed precision to optimize H200 GPU performance
set_global_policy('mixed_float16')

In [ ]:
# Load your dataset
df = pd.read_csv('path/to/card_data.csv')  # Replace with your CSV file path
image_dir = 'path/to/images'  # Replace with your image directory path
df['filename'] = df['filename'].apply(lambda x: os.path.join(image_dir, x))

In [ ]:
# Split into training and validation sets
train_df = df.sample(frac=0.8, random_state=42)
val_df = df.drop(train_df.index)

In [ ]:
# Encode the grade labels (e.g., 'PSA1' to 'PSA10') into integers
le = LabelEncoder()
le.fit(df['grade'])
train_labels = le.transform(train_df['grade'])
val_labels = le.transform(val_df['grade'])

In [ ]:
# Optional: Compute class weights to handle imbalance (remove if not needed)
classes = np.unique(train_df['grade'])
class_weights = compute_class_weight('balanced', classes=classes, y=train_df['grade'])
class_weights_dict = dict(enumerate(class_weights))
train_sample_weights = [class_weights_dict[label] for label in train_labels]

In [ ]:
# Create training dataset with sample weights
train_dataset = tf.data.Dataset.from_tensor_slices((train_df['filename'], train_labels, train_sample_weights))
val_dataset = tf.data.Dataset.from_tensor_slices((val_df['filename'], val_labels))

In [ ]:
# Define image loading and preprocessing function
def load_and_preprocess_image(filename, label, weight=None):
    img = tf.io.read_file(filename)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, [224, 224])  # Resizing to 224x224 as requested
    img = img / 255.0  # Normalize to [0, 1]
    label = tf.one_hot(label, depth=10)  # One-hot encode for 10 classes
    if weight is not None:
        return img, label, weight
    else:
        return img, label

# Define data augmentation function for training
def augment_image(image, label, weight=None):
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_brightness(image, max_delta=0.1)
    # Add more augmentations here if desired (e.g., random rotation)
    if weight is not None:
        return image, label, weight
    else:
        return image, label

In [ ]:
# Configure datasets for performance
batch_size = 64  # Increased from 32; adjust to 128 or higher if H200 memory allows

train_dataset = train_dataset.map(
    lambda x, y, w: load_and_preprocess_image(x, y, w),
    num_parallel_calls=tf.data.AUTOTUNE
)
train_dataset = train_dataset.map(augment_image, num_parallel_calls=tf.data.AUTOTUNE)
train_dataset = train_dataset.shuffle(buffer_size=10000)
train_dataset = train_dataset.batch(batch_size)
train_dataset = train_dataset.prefetch(tf.data.AUTOTUNE)

val_dataset = val_dataset.map(
    load_and_preprocess_image,
    num_parallel_calls=tf.data.AUTOTUNE
)
val_dataset = val_dataset.batch(batch_size)
val_dataset = val_dataset.prefetch(tf.data.AUTOTUNE)

# Build the model with ResNet101
base_model = ResNet101(
    weights='imagenet',
    include_top=False,
    input_shape=(224, 224, 3)
)
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(1024, activation='relu'),
    Dropout(0.5),
    Dense(10, activation='softmax', dtype='float32')  # Output in float32 for stability
])

# Freeze the base model initially
for layer in base_model.layers:
    layer.trainable = False

In [ ]:
# Compile the model
model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Define callbacks
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)
lr_scheduler = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=3,
    min_lr=1e-6
)

In [ ]:
# Train the model (initial phase)
history = model.fit(
    train_dataset,
    epochs=50,
    validation_data=val_dataset,
    callbacks=[early_stopping, lr_scheduler]
)

# Fine-tune by unfreezing the last 10 layers
for layer in base_model.layers[-10:]:
    layer.trainable = True

model.compile(
    optimizer=Adam(learning_rate=0.0001),  # Lower learning rate for fine-tuning
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Fine-tune the model
history_fine = model.fit(
    train_dataset,
    epochs=50,
    validation_data=val_dataset,
    callbacks=[early_stopping, lr_scheduler]
)

In [ ]:
# Optional: Visualize training history
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'] + history_fine.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'] + history_fine.history['val_accuracy'], label='Val Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'] + history_fine.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'] + history_fine.history['val_loss'], label='Val Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()

# Save the trained model
model.save('card_grader_model.h5')

In [ ]:
# Define prediction function
def predict_grade(img_path, model, le):
    img = tf.io.read_file(img_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, [224, 224])
    img = img / 255.0
    img = tf.expand_dims(img, axis=0)
    prediction = model.predict(img)
    predicted_class = np.argmax(prediction, axis=1)[0]
    predicted_grade = le.inverse_transform([predicted_class])[0]
    return predicted_grade

# Example prediction
new_card_path = 'path/to/new/card.jpg'  # Replace with your test image path
predicted_grade = predict_grade(new_card_path, model, le)
print(f"Predicted PSA grade: {predicted_grade}")